# Mass Mobilization Project
<img style="float: right;" src="../images/man_with_hammer.png" />

### 02. Collecting external data, add Prosperity Index data

The Legatum Institute is a London-based think-tank... who worked with more than 100 academics and experts around the world with particular expertise in each of the pillars of prosperity to develop an appropriate taxonomy of discrete elements and supporting indicators which, when combined, accurately capture prosperity in the world.
- https://www.prosperity.com/about/methodology

The 12 pillars created by the Legatum Institute to measure each country are:
 * Safety and Security
 * Personal Freedom
 * Governance
 * Social Capital
 * Investment Environment
 * Enterprise Conditions
 * Market Access & Infrastructure
 * Economic Quality
 * Living Conditions
 * Health
 * Education
 * Natural Environment
 
A lot of work in this notebook is renaming country names in the downloaded Prosperity file to match the names of the protest rows in the Mass Protest data.

Once that is completed, merge the data.

In [1]:
import pandas as pd
import numpy as np

import pickle

In [2]:
mass =pd.read_pickle('../data/02a_added_population.pickle')

mass.head(2)

,id,country,ccode,year,region,protestnumber,startday,startmonth,startyear,endday,...,stateresponse7,sources,notes,protester_id_type,partipants_number,protest_size_category,PopMale,PopFemale,PopTotal,PopDensity
0,201990001,Canada,20,1990,North America,1,15.0,1.0,1990.0,15.0,...,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,workers_unions,1000,"1,000-4,999",13656.649,13884.674,27541.323,3.029
1,201990002,Canada,20,1990,North America,2,25.0,6.0,1990.0,25.0,...,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,students_youth,1000,"1,000-4,999",13656.649,13884.674,27541.323,3.029


In [3]:
print (f'There are { len(set(list(mass["country"]))) } countries in the mass protest dataframe.')

There are 166 countries in the mass protest dataframe.


---
Load Prosperity Index data from first sheet of Prosperity Excel file.

In [4]:
country_PI_df = pd.read_excel('../source/Legatum_Prosperity_Index_Full_2020_Data_Set.xlsx', sheet_name=0)

In [5]:
print (f'The Prosperity Index contains scores on {country_PI_df.shape[0]} countries.')

The Prosperity Index contains scores on 167 countries.


In [6]:
country_PI_df.head(3)

,area_name,area_code,area_group,rank_2007,rank_2008,rank_2009,rank_2010,rank_2011,rank_2012,rank_2013,...,score_2011,score_2012,score_2013,score_2014,score_2015,score_2016,score_2017,score_2018,score_2019,score_2020
0,Afghanistan,AFG,Asia-Pacific,163,163,162,161,162,161,160,...,34.018804,33.529762,34.848416,35.455467,33.878693,34.122529,34.704901,33.234143,33.607822,34.350055
1,Albania,ALB,Eastern Europe,74,75,76,67,66,65,66,...,57.100940,57.345438,57.155699,57.095382,57.409473,57.699371,57.855046,59.182053,60.132551,59.573636
2,Algeria,DZA,Middle East and North Africa,107,109,107,107,103,110,111,...,50.207155,48.905687,48.401319,49.657271,49.944818,50.026566,49.876850,50.919671,51.913637,51.869120


Change dataframe to only contain country name, and the overall Prosperity Index score from 2016 (the most recently calculated score.)

In [7]:
country_PI_df = country_PI_df[['area_name','score_2020']]
country_PI_df.columns = ['country', 'prosperity_2020']

In [9]:
country_PI_df.describe()


,prosperity_2020
count,167.000000
mean,57.507638
std,13.455106
min,27.888813
25%,46.866471
50%,56.881807
75%,66.859066
max,84.368117


### Compare country name in Prosperity Index to Mass Protest data

The country names in the Prosperity Index does not all match the 166 countries in the larger mass protest dataframe.

Loop through both dataframes to identify any discrepancies.

In [92]:
prosperity_countries = sorted(list(country_PI_df['country']))
mass_countries = sorted(list(mass['country'].value_counts().index))

mass_not_prosperity = []
prosperity_not_in_mass = []

for country in mass_countries:
    if country not in prosperity_countries:
        mass_not_prosperity.append(country)
        
for country in prosperity_countries:
    if country not in mass_countries:
        prosperity_not_in_mass.append(country)
        
mass_not_prosperity

['Bhutan',
 'Bosnia',
 'Cape Verde',
 'Congo Brazzaville',
 'Congo Kinshasa',
 'Czech Republic',
 'Czechoslovakia',
 'Gambia',
 'Germany East',
 'Germany West',
 'Ivory Coast',
 'Kosovo',
 'Macedonia',
 'North Korea',
 'Serbia and Montenegro',
 'Slovak Republic',
 'Swaziland',
 'Taiwan',
 'Timor Leste',
 'USSR',
 'United Arab Emirate',
 'Yugoslavia']

In [93]:
prosperity_not_in_mass

['Australia',
 'Belize',
 'Bosnia and Herzegovina',
 'Cabo Verde',
 'Congo',
 'Czechia',
 "Côte d'Ivoire",
 'Democratic Republic of Congo',
 'Eswatini',
 'Hong Kong',
 'Iceland',
 'Israel',
 'Malta',
 'New Zealand',
 'North Macedonia',
 'Seychelles',
 'Slovakia',
 'São Tomé and Príncipe',
 'Taiwan, China',
 'The Gambia',
 'Trinidad and Tobago',
 'United Arab Emirates',
 'United States']

---
**Bhutan**

Bhutan is not included in the Prosperity Index.

From the Legatum Institute [FAQ](https://www.prosperity.com/about/faqs) 

>For some countries in the world there is a lack of available credible data and we have covered over 99% of the world’s population in the Index. To maintain a high level of quality, we only include countries that have data available for at least 50% of our indicators. 

Bhutan has 19 blank protest rows, and only two rows with actual protest data. We could assign the same Prosperity score as Nepal since they are neighboring countries, or just drop.

In [94]:
mass[mass['country']=='Bhutan']['partipants_number'].value_counts()

84     1
100    1
Name: partipants_number, dtype: int64

In [95]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'protesteridentity',
       'protesterdemand1', 'protesterdemand2', 'protesterdemand3',
       'protesterdemand4', 'stateresponse1', 'stateresponse2',
       'stateresponse3', 'stateresponse4', 'stateresponse5', 'stateresponse6',
       'stateresponse7', 'sources', 'notes', 'protester_id_type',
       'partipants_number', 'protest_size_category', 'PopMale', 'PopFemale',
       'PopTotal', 'PopDensity'],
      dtype='object')

Rename Bosnia and Herzegovina to just **Bosnia** to match mass protest dataframe.

In [96]:
country_PI_df[country_PI_df['country']=='Bosnia and Herzegovina']

,country,prosperity_2020
16,Bosnia and Herzegovina,57.998273


In [97]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Bosnia and Herzegovina', 'Bosnia')

**Cape Verde** is also known as Cabo Verde. The mass dataframe contains Cape Verde

In [98]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Cabo Verde', 'Cape Verde')

**Congo**

Mass protest dataframe contains both:
* Congo Brazzaville
* Congo Kinshasa

The Prosperity Index contains values for:
* Congo
* Democratic Republic of the Congo

Since Kinshasa is the capital of the Democratic Republic of the Congo, rename the Prosperity Index label as Congo Kinshasa.

Brazzaville is the capital of the Republic of the Congo, so rename the Prosperity Index label to Congo Brazzaville

In [99]:
country_PI_df.loc[country_PI_df['country'] == 'Congo' , 'country'] = 'Congo Brazzaville'
                
country_PI_df['country'] = country_PI_df['country'].str.replace('Democratic Republic of Congo', 'Congo Kinshasa')

**Czechia**

Mass protest dataframe contains both:

* Czech Republic
* Czechoslovakia
while Prosperity contains a single **Czechia** score.

Perform same operations as to the Congo score.
Assign the single Czechia score to Czech Republic, then create a new row with the same Prosperity score as Czechoslovakia.

In [100]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Czechia', 'Czech Republic')

#Create new row for Czechoslovakia
country_PI_df.loc[country_PI_df.shape[0]] = ['Czechoslovakia' , country_PI_df[country_PI_df['country']=='Czech Republic']['prosperity_2020'].item() ]

In [101]:
country_PI_df.tail(3)

,country,prosperity_2020
165,Zambia,47.509684
166,Zimbabwe,42.977587
167,Czechoslovakia,73.124612


**Gambia** is identified as The Gambia in the Prosperity Index. Rename to simply Gandia to merge correctly with the mass protest dataframe.

In [102]:
country_PI_df['country'] = country_PI_df['country'].str.replace('The Gambia', 'Gambia')

Mass protest dataframe contains protest data for:
* Germany East
* Germany West
* Germany

while Prosperity contains a single **Germany** score.

Create records for East and West Germany scores for a clean merge operation later.

In [103]:
#Create new row for Germany East
country_PI_df.loc[country_PI_df.shape[0]] = ['Germany East' , country_PI_df[country_PI_df['country']=='Germany']['prosperity_2020'].item() ]

#Create new row for Germany West
country_PI_df.loc[country_PI_df.shape[0]] = ['Germany West' , country_PI_df[country_PI_df['country']=='Germany']['prosperity_2020'].item() ]

**Ivory Coast** is identified as Côte d'Ivoire in the Prosperity Index. Rename to merge correctly with the mass protest dataframe.

In [104]:
country_PI_df['country'] = country_PI_df['country'].str.replace("Côte d'Ivoire", 'Ivory Coast')

**Kosovo** is in Mass protest data but has no Prosperity score.

[Independence of Kosovo](https://en.wikipedia.org/wiki/International_recognition_of_Kosovo) is disputed, but it declared independence from Serbia's in 2008.



Assign Serbia's Prosperity score.

In [105]:
#Create new row for Kosovo by copying the score for Serbia.
country_PI_df.loc[country_PI_df.shape[0]] = ['Kosovo' , country_PI_df[country_PI_df['country']=='Serbia']['prosperity_2020'].item() ]

**North Macedonia** was known as Macedonia until February 2019.  Rename North Macedonia in Prosperity Index to Macedonia to match with mass protest dataframe.
https://en.wikipedia.org/wiki/North_Macedonia

In [106]:
country_PI_df['country'] = country_PI_df['country'].str.replace('North Macedonia', 'Macedonia')

**North Korea** does not have a Prosperity score.  From the 2017 Foreward:
'..because of conflict or oppression we do not have the data to report on them.
These include Myanmar, North Korea and Syria'

https://prosperitysite.s3-accelerate.amazonaws.com/3515/1187/1128/Legatum_Prosperity_Index_2017.pdf

Create a score with that is one standard deviation below the mean. Assuming normal distribution this would be approx to 34%

In [107]:
mass[mass['country']=='North Korea']['partipants_number'].value_counts()

50       4
200      2
100      1
30000    1
Name: partipants_number, dtype: int64

North Korea has 8 rows with valid protest data.

In [108]:
country_PI_df.describe()

,prosperity_2020
count,171.000000
mean,57.903588
std,13.591698
min,27.888813
25%,47.111662
50%,57.093573
75%,67.408141
max,84.368117


In [109]:
#Create new row for North Korea
country_PI_df.loc[country_PI_df.shape[0]] = ['North Korea' , country_PI_df['prosperity_2020'].mean() - np.std(country_PI_df['prosperity_2020']) ]

**Serbia and Montenegro**

Mass protest contains protest data for the following:
* Serbia
* Montenegro
* Serbia and Montenegro

The Prosperity Index only contains two scores, one for Serbia and one for Montenegro.  Both values are similar, but create an average of the two for a composite score.

In [110]:
serbia_score = country_PI_df[country_PI_df['country']=='Serbia']['prosperity_2020'].item()
montenegro_score = country_PI_df[country_PI_df['country']=='Montenegro']['prosperity_2020'].item()
avg_prosperity_serb_mont = (serbia_score + montenegro_score) / 2
avg_prosperity_serb_mont

63.07364108225

In [111]:
#Create new row forSerbia and Montenegro
country_PI_df.loc[country_PI_df.shape[0]] = ['Serbia and Montenegro' , avg_prosperity_serb_mont ]

The mass protest dataframe contains values under **'Slovak Republic'** vs 'Slovakia' in the Prosperity Index

In [112]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Slovakia', 'Slovak Republic')

The mass protest dataframe values for **'Swaziland'** are identified as 'Eswatini' in the Prosperity Index

In [113]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Eswatini', 'Swaziland')

Rename Taiwan, China to simply Taiwan

In [114]:
country_PI_df['country'] = country_PI_df['country'].str.replace('Taiwan, China', 'Taiwan')

There is no Prosperity Index for **Timor-Leste**, also known as East Timor. Became a sovereign state in 2002 after being under Indonesian rule. Accounts for 0.02% of world's population.  22 rows in mass protest data, 7 of which are nan. We could drop or just assign the same score as Indonesia.

In [115]:
mass[mass['country']=='Timor Leste'].shape

(12, 35)

In [116]:
#Create new row for Timor Leste
country_PI_df.loc[country_PI_df.shape[0]] = ['Timor Leste' , country_PI_df[country_PI_df['country']=='Indonesia']['prosperity_2020'].item() ]

In [117]:
mass.shape

(15227, 35)

Use same Prosperity score for USSR as Russia.

In [118]:
#Create new row for USSR
country_PI_df.loc[country_PI_df.shape[0]] = ['USSR' , country_PI_df[country_PI_df['country']=='Russia']['prosperity_2020'].item() ]

The mass protest dataframe values for 'United Arab Emirate' are identified as 'United Arab Emirates' in the Prosperity Index

In [119]:
country_PI_df['country'] = country_PI_df['country'].str.replace('United Arab Emirates', 'United Arab Emirate')

**Yugoslavia** has no current Prosperity score, but could be determined by former countries that made up [Yugoslavia](https://en.wikipedia.org/wiki/Yugoslavia)

* Bosnia
* Croatia
* Macedonia
* Montenegro
* Serbia
* Slovenia

In [120]:
country_PI_df[country_PI_df['country'].str.startswith('Sl')]

,country,prosperity_2020
131,Slovak Republic,69.625246
132,Slovenia,74.563150


In [121]:
yugo_countries = ['Bosnia', 'Croatia', 'Macedonia', 'Montenegro', 'Serbia', 'Slovenia']
yugo_mean = np.mean([country_PI_df[country_PI_df['country']==country]['prosperity_2020'].item() for country in yugo_countries])
yugo_mean

64.38861633025

In [122]:
#Create new row for Yugoslavia
country_PI_df.loc[country_PI_df.shape[0]] = ['Yugoslavia' , yugo_mean ]

---
Double check that there aren't any countries in the mass protest dataframe that do not have a matching Prosperity Index score

In [123]:
prosperity_countries = sorted(list(country_PI_df['country']))
mass_countries = sorted(list(mass['country'].value_counts().index))

mass_not_prosperity = []
prosperity_not_in_mass = []

for country in mass_countries:
    if country not in prosperity_countries:
        mass_not_prosperity.append(country)
        
for country in prosperity_countries:
    if country not in mass_countries:
        prosperity_not_in_mass.append(country)
        
mass_not_prosperity

['Bhutan']

In [124]:
prosperity_not_in_mass

['Australia',
 'Belize',
 'Hong Kong',
 'Iceland',
 'Israel',
 'Malta',
 'New Zealand',
 'Seychelles',
 'São Tomé and Príncipe',
 'Trinidad and Tobago',
 'United States']

Drop all rows from Prosperity Index that are not in the mass protest dataframe.

In [125]:
#sorted(list(country_PI_df['country']))

In [126]:
for country_to_drop in prosperity_not_in_mass:
    country_PI_df.drop( country_PI_df[country_PI_df['country'] == country_to_drop].index, inplace=True)


In [127]:
len(list(country_PI_df['country']))

165

In [128]:
len(set(list(mass['country'])))

166

In [129]:
### Save Prosperity Index data as csv for merging into other notebook

In [130]:
# country_PI_df.to_csv('../data/prosperity_clean.csv')

**Bhutan** has 31 rows in Mass protest, but only two that are not all NaN.
Use merge that will drop the Bhutan columns

In [131]:
mass[mass['country']=='Bhutan'].shape

(2, 35)

In [132]:
# Used to spot check out a country's mass protest data
#mass[mass['country']=='Bhutan'][['partipants_number','startyear','endmonth','endyear','protesterdemand1','stateresponse1']]

**Merge** into larger mass protest dataframe.

In [133]:
# Doing a merge with the defaults will drop rows in mass that do not have a prosperity score (31 rows of Bhutan) 
mass = mass.merge(country_PI_df)

### Save updated Mass Protest dataframe to pickle object to hand off to next stage.

In [134]:
mass.to_pickle('../data/02b_added_prosperity.pickle')

---

In [135]:
# Doing a merge with this command will keep Bhutan and insert a NaN in its prosperity_2020 column
# mass = mass.merge(country_PI_df, how='left', 
#                         left_on='country', 
#                         right_on='country')
# mass.head(2)

In [136]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protestnumber', 'startday',
       'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'protesteridentity',
       'protesterdemand1', 'protesterdemand2', 'protesterdemand3',
       'protesterdemand4', 'stateresponse1', 'stateresponse2',
       'stateresponse3', 'stateresponse4', 'stateresponse5', 'stateresponse6',
       'stateresponse7', 'sources', 'notes', 'protester_id_type',
       'partipants_number', 'protest_size_category', 'PopMale', 'PopFemale',
       'PopTotal', 'PopDensity', 'prosperity_2020'],
      dtype='object')